# Overview

This notebook demonstrates a **smart assistant** for analyzing text documents using the **LangChain** framework. It extracts text from uploaded PDF documents, splits the content into manageable chunks, and creates a vector database for efficient semantic search and question answering. The assistant utilizes **Hugging Face embeddings** for semantic similarity and FAISS for vector storage. It is designed to answer questions based on the contents of the provided document while ignoring unrelated queries.

### Key Features:
- **Dynamic Input**: Upload any PDF document for analysis.
- **Semantic Search**: Retrieve relevant information using advanced embeddings.
- **Efficient Storage**: Utilize FAISS for scalable vector-based storage.

### Example Data:
This implementation uses the document [**"Text Classification Algorithms"**](https://www.mdpi.com/2078-2489/10/4/150?source=post_page---------------------------), which provides insights into various methods and steps involved in text classification.

### Use Cases:
- Analyze scientific papers, reports, or articles.
- Retrieve specific details from large documents.
- Dynamically update the assistant's knowledge base by uploading new PDFs.

Feel free to upload any PDF document to test and customize the assistant for your specific needs.


In [1]:
!pip install langchain
!pip install -U langchain-community
!pip install pdfplumber
!pip install faiss-gpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 58.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 4.6 MB/s eta 0:00:00
  Attempting uninstall: SQLAlchemy
    Found existing installation: SQLAlchemy 2.0.36
    Uninstalling SQLAlchemy-2.0.36:
      Successfully uninstalled SQLAlchemy-2.0.36
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.2/59.2 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 106.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 8.3 MB/s eta 0:00:00


### Import libraries

In [14]:
import pdfplumber
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain.llms import OpenAI

### Upload and process PDF documents

In [3]:
def extract_text_from_pdf(file_path):
    with pdfplumber.open(file_path) as pdf:
        text = ''
        for page in pdf.pages:
            text += page.extract_text()
    return text

In [7]:
text = extract_text_from_pdf("Text Classification Algorithms.pdf")

### Splitting text into parts with LangChain

In [8]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks = text_splitter.split_text(text)

In [9]:
chunks[0]

'Review\nText Classification Algorithms: A Survey\nKamranKowsari1,2,* ID,KianaJafariMeimandi1,MojtabaHeidarysafa1,SanjanaMendu1 ID,\nLauraBarnes1,2,3 ID andDonaldBrown1,3 ID\n1 DepartmentofSystemsandInformationEngineering,UniversityofVirginia,Charlottesville,VA22904,USA;\nkj6vd@virginia.edu(K.J.M.);mh4pk@virginia.edu(M.H.);sm7gc@virginia.edu(S.M.);\nlb3dp@virginia.edu(L.B.);deb@virginia.edu(D.B.)\n2 SensingSystemsforHealthLab,UniversityofVirginia,Charlottesville,VA22911,USA\n3 SchoolofDataScience,UniversityofVirginia,Charlottesville,VA22904,USA\n* Correspondence:kk7nc@virginia.edu;Tel.:+1-202-812-3013\n(cid:1)(cid:2)(cid:3)(cid:1)(cid:4)(cid:5)(cid:6)(cid:7)(cid:8)(cid:1)\n(cid:1)(cid:2)(cid:3)(cid:4)(cid:5)(cid:6)(cid:7)\nReceived:22March2019;Accepted:17April2019;Published:23April2019\nAbstract: Inrecentyears,therehasbeenanexponentialgrowthinthenumberofcomplexdocuments\nandtextsthatrequireadeeperunderstandingofmachinelearningmethodstobeabletoaccurately'

### Creating a vector database using LangChain

In [10]:
def create_vector_database(chunks):
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
  vector_database = FAISS.from_texts(chunks, embedding=embeddings)
  return vector_database

In [11]:
vector_database = create_vector_database(chunks)
type(vector_database)

<ipython-input-10-56e7e2fcafeb>:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a toke

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

langchain_community.vectorstores.faiss.FAISS

### Search with LangChain

In [17]:
import os
os.environ["OPENAI_API_KEY"] = "your OpenAI API key here..."

In [18]:
def search_answer(query, vector_database):
  retriever = vector_database.as_retriever()
  chain = RetrievalQA.from_chain_type(llm=OpenAI(), chain_type="stuff", retriever=retriever)
  answer = chain.run(query)
  return answer

### Test how it works

In [22]:
query = "what is text classification?"
answer = search_answer(query, vector_database)
print(answer)

 Text classification is the process of automatically categorizing or organizing text documents into different groups or categories based on their content. It involves using machine learning algorithms to analyze and extract features from text data, reducing its dimensionality, selecting a classifier, and evaluating its performance. Text classification is commonly used in various fields such as information retrieval, medicine, social sciences, healthcare, psychology, and law. 


In [24]:
query = "what are the main steps of text classification you know?"
answer = search_answer(query, vector_database)
print(answer)

 The main steps of text classification include feature extraction, dimension reduction, classifier selection, and evaluation.


In [25]:
query = "tell me about Rocchio Classification"
answer = search_answer(query, vector_database)
print(answer)

 The Rocchio algorithm was first introduced in 1971 by J.J. Rocchio as a method for using relevance feedback to query full-text databases. It is a classification algorithm that uses TF-IDF weights for each informative word instead of boolean features, and assigns each test document to the class with the maximum similarity between the test document and each prototype vector. Some advantages of this algorithm include easy implementation and low computational complexity, but it can also misclassify multi-modal classes and is not very robust. It also requires careful tuning of hyper-parameters. 


In [26]:
query = "what is the weather in New York in the summer?"
answer = search_answer(query, vector_database)
print(answer)

 I don't know.


### Conclusion
The smart assistant successfully answered all questions related to the contents of the provided document, such as "What is text classification?" and "Tell me about Rocchio Classification." However, when asked about unrelated topics, such as "What is the weather in New York in the summer?", the assistant correctly responded with "I don't know", as this information was not included in the document. This demonstrates the system's ability to provide accurate and document-specific responses while maintaining contextual relevance.